<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/TruncatedSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Final/soynlp/newsresult.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
doc_ls = df['제목']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

n_topics = 4

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(doc_ls)
svd = TruncatedSVD(n_components=n_topics, n_iter=100)

U = svd.fit_transform(tfidf)
Vt = svd.components_

In [ ]:
vocab = tfidf_vect.get_feature_names_out()
print(len(vocab))

36505


In [ ]:
n = 4
for i, topic in enumerate(Vt):
    print("Topic {} :".format(i), [ (vocab[i], topic[i].round(3)) for i in topic.argsort()[:-n-1:-1]])

Topic 0 : [('결정', 0.578), ('현금배당', 0.552), ('주당', 0.475), ('100원', 0.113)]
Topic 1 : [('지원', 0.45), ('러시아', 0.195), ('우크라', 0.194), ('최대', 0.194)]
Topic 2 : [('3월', 0.378), ('코스피', 0.192), ('상승', 0.178), ('하락', 0.16)]
Topic 3 : [('3월', 0.563), ('지원', 0.276), ('동향', 0.13), ('증시', 0.129)]


In [ ]:
n = 3
temp = [0, 1, 2, 3, 4, 5]
temp[:-n:-1]

[5, 4]

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

def calc_similarity_matrix(vectors):
    
    n_word = len(vectors)
    similarity_matrix = np.zeros((n_word, n_word))
    
    for i in range(n_word):
        for j in range(n_word):
            similarity_matrix[i, j] = cosine_similarity(vectors[i], vectors[j]).round(3)
            
    return similarity_matrix

In [ ]:
vectors = Vt.T
word_similarity_matrix = calc_similarity_matrix(vectors)
word_similarity_matrix.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font", family="Malgun Gothic") #windows
# plt.rc("font", family="AppleGothic") #mac

def visualize_similarity(similarity_matrix, label):
    mask = np.triu(np.ones_like(similarity_matrix, dtype=np.bool))
    plt.rcParams['figure.figsize'] = [8, 6]
    ax = sns.heatmap(similarity_matrix, mask=mask, xticklabels=label, yticklabels=label, 
                     annot=True, fmt=".2f", annot_kws={"size":8}, cmap="coolwarm") 

In [ ]:
visualize_similarity(word_similarity_matrix, vocab)

In [ ]:
doc_similarity_matrix = calc_similarity_matrix(U)
visualize_similarity(doc_similarity_matrix, doc_ls)

In [ ]:
from sklearn.manifold import TSNE

def visualize_vectors(vectors, labels):
    tsne = TSNE(n_components=2, n_iter=10000, perplexity=2)
    T = tsne.fit_transform(vectors)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
    for label, x, y in zip(labels, T[:, 0], T[:, 1]):
        plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords="offset points")

In [ ]:
visualize_vectors(vectors, vocab)

In [ ]:
!pip install pyLDAvis

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(tokenized_docs)
lda = LatentDirichletAllocation(n_components=5, max_iter=50, learning_method="online", random_state=42)
lda_output = lda.fit_transform(tfidf)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds="tsne")
pyLDAvis.display(vis)